In [6]:
import numpy as np
import pandas as pd
import json
import keras
from keras import layers
import random
import sys

In [7]:
# read in reviews
reviews = pd.read_json('./data/Office_Products_5.json',lines=True)

In [8]:
# drop non-verified
reviews.drop(reviews[reviews['verified'] == False].index, inplace = True)

In [9]:
reviews.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"11 7, 2017",A2NIJTYWADLK57,0140503528,{'Format:': ' Board book'},cotton clay,kids like story BUT while i really wanted a bo...,"good story, small size book though",1510012800,NaN,NaN
1,4,True,"03 7, 2017",A2827D8EEURMP4,0140503528,{'Format:': ' Hardcover'},emankcin,Bought this used and it came in great conditio...,Good,1488844800,NaN,NaN
2,5,True,"06 25, 2016",APB6087F4J09J,0140503528,{'Format:': ' Board book'},Starbucks Fan,Every story and book about Corduroy is Fantast...,Best Books for All Children,1466812800,NaN,NaN
3,5,True,"02 21, 2016",A2DHERRZIPFU7X,0140503528,{'Format:': ' Paperback'},Caitlyn Jacobson,I purchased this book for my first grade class...,Great for Math!,1456012800,NaN,NaN
5,5,True,"01 1, 2015",A3JJTLTBCE7T7L,0140503528,{'Format:': ' Paperback'},Maestra Mam,Corduroy gets a pocket....riveting stuff people.,Five Stars,1420070400,NaN,NaN


In [10]:
reviews[reviews['overall']==5]['reviewText'].str.len().describe()

count    517305.000000
mean        162.116595
std         287.486511
min           1.000000
25%          28.000000
50%          79.000000
75%         185.000000
max       16583.000000
Name: reviewText, dtype: float64

In [24]:
reviews['reviewText'].str.len().describe()

count    709987.000000
mean        189.383886
std         330.323343
min           1.000000
25%          32.000000
50%          96.000000
75%         217.000000
max       32602.000000
Name: reviewText, dtype: float64

In [25]:
one_star_sample = reviews[(reviews['overall']==1) & (reviews['reviewText'].str.len() > 99)
        & (reviews['reviewText'].str.len() < 301)].sample(frac=.1, replace=False, random_state=17)

two_star_sample = reviews[(reviews['overall']==2) & (reviews['reviewText'].str.len() > 99)
        & (reviews['reviewText'].str.len() < 301)].sample(frac=.1, replace=False, random_state=17)

three_star_sample = reviews[(reviews['overall']==3) & (reviews['reviewText'].str.len() > 99)
        & (reviews['reviewText'].str.len() < 301)].sample(frac=.1, replace=False, random_state=17)

four_star_sample = reviews[(reviews['overall']==4) & (reviews['reviewText'].str.len() > 99)
        & (reviews['reviewText'].str.len() < 301)].sample(frac=.1, replace=False, random_state=17)

five_star_sample = reviews[(reviews['overall']==5) & (reviews['reviewText'].str.len() > 99)
        & (reviews['reviewText'].str.len() < 301)].sample(frac=.1, replace=False, random_state=17)

review_sample = pd.concat([one_star_sample, two_star_sample, three_star_sample,
          four_star_sample, five_star_sample], ignore_index=True)

In [26]:
review_sample

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,1,True,"08 1, 2017",A1Q8QKKXDN0E3G,B00CC76756,{'Color:': ' Silver'},T. Scott,"Leaks, squirts out ink randomly, not an even f...",Horrid,1501545600,NaN,NaN
1,1,True,"04 14, 2016",A39GX54LTJ1BFX,B00EM4S34K,NaN,J-nice,"Love the pen size, 0.38 but the ink ran out VE...",Frustrated with pens running out of ink QUICKLY,1460592000,NaN,NaN
2,1,True,"11 21, 2013",A3HLH7UOHXEX94,B0007KT29I,"{'Size:': ' 18""x6''}",Syl Flower,"This product didn;t stick to surface, no matte...",Poor performing item.,1384992000,NaN,NaN
3,1,True,"04 14, 2015",A1CQWEPJ2GH1I6,B0035YI8WA,NaN,Kathy,I received a plastic tote with no wheels inclu...,Received defective product,1428969600,NaN,NaN
4,1,True,"01 6, 2016",A9EVZVHSVO6NS,B000OZLRHQ,NaN,Aaron Q.,They clog up the printer really badly . Had to...,They clog up the printer really badly. Had to ...,1452038400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
22578,5,True,"01 11, 2016",A380WUQNSD2SQM,B006Q20DM6,NaN,Amazon Customer,These are my favorite pencils on the market. I...,5 stars,1452470400,NaN,NaN
22579,5,True,"03 29, 2013",A1XNHYSEF0TU57,B0030BERUK,{'Color:': ' Red'},osk,"One thing for sure, one is not enough. When tr...",Love the product!,1364515200,NaN,NaN
22580,5,True,"02 23, 2016",A18YMF9QF0SA5P,B00KY10YWO,NaN,Rose,Great deal. Much easier to buy online and have...,Great deal. Much easier to buy online and have...,1456185600,NaN,NaN
22581,5,True,"11 2, 2015",A3RR3UT7XS6FBX,B00006IFJ0,"{'Size:': ' 12-Count', 'Color:': ' Assorted Co...",Sedgewick Collins,This was a very nice set for the price that I...,Small Town Man Uses Markers To Talk With Alien...,1446422400,NaN,NaN


In [27]:
corpus = review_sample['reviewText'].str.cat(sep='\n')

In [28]:
print(f'Corpus length: {len(corpus)}')

Corpus length: 3977962


In [38]:
# extract sequences of 45 characters
maxlen = 25

# sample new sequence every 3 characters
step = 3

# hold extracted sequences
sentences = []

# hold targets
next_chars = []

In [39]:
for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i: i + maxlen])
    next_chars.append(corpus[i + maxlen])
    
print('Number of sequences:', len(sentences))

Number of sequences: 1325979


In [40]:
# sorted list of unique characters in text
chars = sorted(list(set(corpus)))
print('Unique characters:', len(chars))

Unique characters: 96


In [41]:
# dictionary that maps unique characters to index in list {chars}
char_indices = dict((char, chars.index(char)) for char in chars)

In [42]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [43]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [44]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [45]:
def sample(preds, temparature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [46]:
for epoch in range(1, 60):
    print(f'epoch {epoch}')
    model.fit(x, y, batch_size=128, epochs=1)
    
    # open a file to write results to for certain epochs
    with  open(f"./results/samples3_epoch_{epoch}.txt","w") as outFile:
        # select text seed at random
        start_index = random.randint(0, len(corpus) - maxlen - 1)
        seed_text = corpus[start_index: start_index + maxlen]

        outFile.write(f'\n--- Generating with seed:"{seed_text}"\n')
        print(f'\n--- Generating with seed:"{seed_text}"\n')

        for temperature in [0.2, 0.5, 1.0, 1.2]:
            outFile.write(f'\n------ temperature: {temperature}\n')
            print(f'\n------ temperature: {temperature}\n')

            for i in range(200):
                sampled = np.zeros((1, maxlen, len(chars)))

                # one hote encode chracters generated so far
                for t, char in enumerate(seed_text):
                    sampled[0, t, char_indices[char]] = 1

                # predict and sample the next characther
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                    
                next_char = chars[next_index]

                seed_text += next_char
                seed_text = seed_text[1:]

                outFile.write(next_char)
    

epoch 1
10360/10360 [==============================] - 339s 33ms/step - loss: 1.5264

--- Generating with seed:"wished the leather was th"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 2
10360/10360 [==============================] - 335s 32ms/step - loss: 1.37562s -

--- Generating with seed:"vy-duty and substantial. "


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 3
10360/10360 [==============================] - 329s 32ms/step - loss: 1.35040s - loss: 1

--- Generating with seed:"ng is nicely contained.
E"


------ temperature: 0.2



<ipython-input-45-c17b164e6a93>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature



------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 4
10360/10360 [==============================] - 336s 32ms/step - loss: 1.3400

--- Generating with seed:" Because they are gel pen"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 5
10360/10360 [==============================] - 330s 32ms/step - loss: 1.3349

--- Generating with seed:"ce supply store. Good qua"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 6
10360/10360 [==============================] - 326s 31ms/step - loss: 1.3330

--- Generating with seed:"round the house, works gr"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 7
10360/10360 [==============================] - 330s 32ms/step - loss: 1.3364

--- Generating with seed:"perwork and it suit my ne"


------ temperature: 0.2


------ temperature: 0.5

epoch 37
10360/10360 [==============================] - 307s 30ms/step - loss: 3.1283

--- Generating with seed:"being too close to the ce"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 38
10360/10360 [==============================] - 306s 30ms/step - loss: 3.1295

--- Generating with seed:" Return and was returned "


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 39
10360/10360 [==============================] - 308s 30ms/step - loss: 3.1346

--- Generating with seed:"y brand I will use. And t"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 40
10360/10360 [==============================] - 305s 29ms/step - loss: 3.1376

--- Generating with seed:"en is a vibrant & true gr"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 41
10360/10360